<a href="https://colab.research.google.com/github/robd123/IndepResearch/blob/master/BugClassificationT5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_name = "t5-small"
token_len = 512
model_prefix = f"{model_name}-{token_len}"

In [2]:
pip install sentencepiece

     |████████████████████████████████| 1.2 MB 12.1 MB/s 


In [3]:
pip install pytorch-lightning

     |████████████████████████████████| 527 kB 11.7 MB/s 
     |████████████████████████████████| 397 kB 45.7 MB/s 
     |████████████████████████████████| 596 kB 37.8 MB/s 
     |████████████████████████████████| 952 kB 6.6 MB/s 
     |████████████████████████████████| 134 kB 49.0 MB/s 
     |████████████████████████████████| 829 kB 45.0 MB/s 
     |████████████████████████████████| 1.1 MB 32.1 MB/s 
     |████████████████████████████████| 94 kB 2.2 MB/s 
     |████████████████████████████████| 144 kB 41.3 MB/s 
     |████████████████████████████████| 271 kB 16.2 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=5f3d73acf3be762fa0987243ce196bc163f59900aca46b89e3e330249c2f966d
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successful

In [4]:
pip install nlp

     |████████████████████████████████| 1.7 MB 12.3 MB/s 
     |████████████████████████████████| 243 kB 47.0 MB/s 


In [5]:
pip install transformers

     |████████████████████████████████| 3.5 MB 13.8 MB/s 
     |████████████████████████████████| 895 kB 30.5 MB/s 
     |████████████████████████████████| 67 kB 5.6 MB/s 
     |████████████████████████████████| 6.8 MB 47.4 MB/s 


In [6]:
pip install wandb

     |████████████████████████████████| 1.7 MB 12.3 MB/s 
     |████████████████████████████████| 181 kB 48.4 MB/s 
     |████████████████████████████████| 144 kB 34.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=1c47aa5a4ad47ac808d48ee9a87326189a3cd326c61ae5420509b55c0ce4e1bb
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [7]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
import torch
from pytorch_lightning.loggers.wandb import WandbLogger
import os
from pathlib import Path
from string import punctuation

os.environ["WANDB_API_KEY"] = '33461c97826e5a650bb71bec8b3e694e7c6ee9f6'
wandb_logger = WandbLogger(project='bugclassification-t5small')

In [9]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import string
import pandas as pd
# pd.set_option('display.max_colwidth', -1)
import numpy as np
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.loggers import WandbLogger
from nlp import load_metric

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)
import random
import re


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
from google.colab import auth
auth.authenticate_user()

!gcloud config set project eternal-bruin-340122

!gsutil cp gs://robd_indep_research/linux_bugs_usage_ready.csv /tmp/dataset.csv

Updated property [core/project].
Copying gs://robd_indep_research/linux_bugs_usage_ready.csv...
- [1 files][ 40.8 MiB/ 40.8 MiB]                                                
Operation completed over 1 objects/40.8 MiB.                                     


Text Processing Functions
From https://github.com/Forethought-Technologies/ieee-dsmp-2018-paper/blob/master/text_processing.py

In [14]:
import string
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

def strip_punctuations(data, column_name='text'):
  '''
  Strips punctuations from the end of each token.
  This uses suggestion from https://stackoverflow.com/questions/34293875/how-to-remove-punctuation-marks-from-a-string-in-python-3-x-using-translate
  to accomplish this really fast.
  '''
  translator = str.maketrans('', '', string.punctuation)
  data['text'] = data['text'].map(lambda s : str(s).translate(translator))
  return data

def stemm_text(data, stemmer_choice='Lancaster'):
  '''
  Stemm the 'text' column of data - this simplifies the words so
  that different forms of the same word end up being the same.
  '''
  if stemmer_choice == 'Lancaster':
    stemmer = LancasterStemmer()
  elif stemmer_choice == 'Snowball':
    stemmer = SnowballStemmer('english')
  elif stemmer_choice == 'Porter':
    stemmer = PorterStemmer()
  else:
    raise Exception('Illegal stemmer_choice argument')
  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: stemmer.stem(w), s.split())))
  return data

def remove_long_words(data, maxlen=16):
  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if len(w) < maxlen else ' ', s.split())))
  return data

def remove_short_words(data, minlen=4):
  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if len(w) >= minlen else ' ', s.split())))
  return data

def remove_linux_garbage(data):
  '''
  Linux data contains lots of garbage, e.g. memory addresses - 0000f800
  '''
  def is_garbage(w):
    return len(w) >= 7 and sum(c.isdigit() for c in w) >= 2

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if not is_garbage(w) else ' ', s.split())))
  return data

def cast_to_lowercase(data):
  data['text'] = data['text'].map(lambda s : s.lower())
  return data

def remove_stopwords(data):
  stop_words = stopwords.words('english')
  translator = str.maketrans('', '', string.punctuation)
  stop_words = set([w.translate(translator) for w in stop_words]) # Apostrophes were removed already

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if w not in stop_words else ' ', s.split())))
  return data

def remove_rare_words(data, min_count=3):
  wc = {} # WordCount
  def proc_word(s):
    for w in set(s.split()):
      if w in wc:
        wc[w] += 1
      else:
        wc[w] = 1

  for index, row in data.iterrows():
    proc_word(row['text'])

  data['text'] = data['text'].map(lambda s : ' '.join(map(lambda w: w if wc[w] >= min_count else ' ', s.split())))
  return data

In [12]:
data = pd.read_csv('/tmp/dataset.csv', sep='\t')

In [13]:
data.head()

,title,message,importance,product,component
0,oops when using ide-cd with 2.5.45 and cdrecord,Please enter Exact Kernel version:2.5.45 Distr...,P2 normal,IO/Storage,IDE
1,NUMA-Q hangs during TSC initialization on boot.,Exact Kernel version: 2.5.46 Distribution: deb...,P2 normal,Platform Specific/Hardware,i386
2,Enabling shared pagetables causes KDE to wierd...,Exact Kernel version: 2.5.46-mm1 Distribution:...,P2 normal,Memory Management,Other
3,Dcache spirals out of control on 2.5.43-mm2,Exact Kernel version: 2.5.43-mm2 Distribution:...,P2 blocking,IO/Storage,Other
4,64GB highmem BUG(),Exact Kernel version: 2.5.40 Hardware Environm...,P2 normal,Memory Management,Other


In [15]:
  data['text'] = data['title'] + ' ' + data['message']
  data = data.drop(['title'], axis=1)
  data = data.drop(['message'], axis=1)
  data.head()

In [18]:
nltk.download('stopwords')
data = strip_punctuations(data)
data = remove_linux_garbage(data)
data = cast_to_lowercase(data)
data = remove_stopwords(data)
data =remove_rare_words(data)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [19]:
data.head()

,importance,product,component,text
0,P2 normal,IO/Storage,IDE,oops using idecd 2545 cdrecord please enter ex...
1,P2 normal,Platform Specific/Hardware,i386,numaq hangs tsc initialization boot exact kern...
2,P2 normal,Memory Management,Other,enabling shared pagetables causes kde wierd ex...
3,P2 blocking,IO/Storage,Other,dcache control exact kernel version distribu...
4,P2 normal,Memory Management,Other,64gb highmem bug exact kernel version 2540 har...


In [20]:
data.shape

(16456, 4)

In [40]:
import numpy as np
import pandas as pd


class BugData(Dataset):
    def __init__(self, df, tokenizer, type_path, num_samples, input_length=4096, output_length=4096, print_text=False):         
        ## Shuffle data set
        df = df.sample(frac=1, random_state=1)
        self.dataset = []
        val_size = int(0.2 * df.shape[0])

        inps = df["text"].values.tolist()
        lbls = df["importance"].values.tolist()

        for i, inp in enumerate(inps):
            inp = inp.replace(":", "")
            inp = "classify: " + inp
            lbl = str(lbls[i])
            self.dataset.append({"inp": inp, "lbl": lbl})
        
        if type_path == "train":
            self.dataset = self.dataset[:len(self.dataset) - val_size - val_size]
        elif type_path == "validation":
            self.dataset = self.dataset[len(self.dataset) - val_size - val_size: len(self.dataset) - val_size]
        elif type_path == "test":
            self.dataset = self.dataset[len(self.dataset) - val_size:]

        if num_samples:
            self.dataset = self.dataset[:num_samples]
        
        self.input_length = input_length
        self.tokenizer = tokenizer
        self.output_length = output_length
        self.print_text = print_text
        
  
    def __len__(self):
        return len(self.dataset)

    def convert_to_features(self, example_batch):
        
        input_ = example_batch['inp']
        target_ = example_batch['lbl']
    
        
        input_ = input_.strip()

        
        source = self.tokenizer.batch_encode_plus([input_], max_length=self.input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
        
        targets = self.tokenizer.batch_encode_plus([target_], max_length=self.input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")

        return source, targets
  
    def __getitem__(self, index):
        source, targets = self.convert_to_features(self.dataset[index])
        
        if self.print_text:
            print("Lens are: ", source['input_ids'][0].shape, targets['input_ids'][0].shape)
        
        source_ids = source["input_ids"].squeeze()
        target_ids = targets["input_ids"].squeeze()

        # return {"source_ids": source_ids, "target_ids": target_ids}

        src_mask    = source["attention_mask"].squeeze()
        target_mask = targets["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}



def get_dataset(tokenizer, type_path, num_samples, args, df=data):
      return BugData(df = df, tokenizer=tokenizer, type_path=type_path, num_samples=num_samples, input_length=args.max_input_length, 
                        output_length=args.max_output_length)

In [41]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
dataset = BugData(data, tokenizer, 'validation', None, token_len, 512, True)
len(dataset)

3291

In [42]:
data_sc = dataset[3]
print()
print("Shape of Tokenized Text: ", data_sc['source_ids'].shape)
print()
print("Sanity check - Decode Text: ", tokenizer.decode(data_sc['source_ids']))
print("====================================")
print("Sanity check - Decode Classification: ", tokenizer.decode(data_sc['target_ids']))

Lens are:  torch.Size([512]) torch.Size([512])

Shape of Tokenized Text:  torch.Size([512])

Sanity check - Decode Text:  classify: missing struct definition moduleh compiling module written get compile error dereferencing pointer incomplete type reason ‘struct includelinuxmoduleh member struct struct can’t dereferenced writing module defined kernelmodulec intended hide definition bug</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

In [43]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix((remove_punc(lower(s))))

In [25]:
def exact_match_score(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

In [26]:
def calculate_scores(predictions, ground_truths):
    em_score = 0
    subset_match_score = 0
    
    for i in range(len(predictions)):
        ground_truth = ground_truths[i]
        prediction = predictions[i]
        em_score +=  exact_match_score(prediction, ground_truth)
    
    em_score /= len(predictions)
    return em_score*100

In [44]:
class T5BugClassifier(pl.LightningModule):
    def __init__(self, hparams):
        super(T5BugClassifier, self).__init__()
        self.save_hyperparameters(hparams)        
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path, return_dict=True)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, max_length=hparams.max_input_length)
        self.output_dir = Path(self.hparams.output_dir)
        self.step_count = 0
        
        if self.hparams.freeze_embeds:
            self.freeze_embeds()
        if self.hparams.freeze_encoder:
            self.freeze_params(self.model.get_encoder())
            assert_all_frozen(self.model.get_encoder())
            
            
        n_observations_per_split = {
            "train": self.hparams.n_train,
            "validation": self.hparams.n_val,
            "test": self.hparams.n_test,
        }
        self.n_obs = {k: v if v >= 0 else None for k, v in n_observations_per_split.items()}
        self.em_score_list = []
        
    def freeze_params(self, model):
        for par in model.parameters():
            par.requires_grad = False
            
    def freeze_embeds(self):
        try:
            self.freeze_params(self.model.model.shared)
            for d in [self.model.model.encoder, self.model.model.decoder]:
                freeze_params(d.embed_positions)
                freeze_params(d.embed_tokens)
        except AttributeError:
            self.freeze_params(self.model.shared)
            for d in [self.model.encoder, self.model.decoder]:
                self.freeze_params(d.embed_tokens)

    def lmap(self, f, x):
        return list(map(f, x))

    def is_logger(self):
        return self.trainer.global_rank <= 0
    
    def parse_score(self, result):
        return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
        
    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None):
        return self.model(
                input_ids,
                attention_mask=attention_mask,
                decoder_input_ids=decoder_input_ids,
                decoder_attention_mask=decoder_attention_mask,
                labels=lm_labels
            )

    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            lm_labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss
    
    def ids_to_clean_text(self, generated_ids):
        gen_text = self.tokenizer.batch_decode(
            generated_ids,
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=True
        )
        return self.lmap(str.strip, gen_text)
    
    def _generative_step(self, batch) :
        
        t0 = time.time()
        # print(batch)
        inp_ids = batch["source_ids"]
        
        generated_ids = self.model.generate(
            batch["source_ids"],
            attention_mask=batch["source_mask"],
            use_cache=True,
            decoder_attention_mask=batch['target_mask'],
            max_length=3

        )
        preds = self.ids_to_clean_text(generated_ids)
        target = self.ids_to_clean_text(batch["target_ids"])
#         print("Preds and Targets: ", preds, target)
            
        gen_time = (time.time() - t0) / batch["source_ids"].shape[0]  
    
        loss = self._step(batch)
        base_metrics = {'val_loss': loss}
        summ_len = np.mean(self.lmap(len, generated_ids))
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target)
        em_score  = calculate_scores(preds, target)
        
        self.em_score_list.append(em_score)
        
        em_score = torch.tensor(em_score,dtype=torch.float32)
    
        base_metrics.update(accuracy=em_score)
        self.log("val/em_score", em_score, on_step=False, on_epoch=True)
        self.log("val/loss", loss, on_step=False, on_epoch=True)
        return base_metrics

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)
        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}
  
    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}

    def validation_step(self, batch, batch_idx):
        return self._generative_step(batch)
    
    def validation_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        
        if len(self.em_score_list) <= 2:
            average_em_score = sum(self.em_score_list) / len(self.em_score_list) 
            
        else:
            latest_em_score = self.em_score_list[:-2]
            average_em_score = sum(latest_em_score) / len(latest_em_score) 
        
        average_em_score = torch.tensor(average_em_score,dtype=torch.float32)
        tensorboard_logs.update(accuracy=average_em_score)
        
        self.target_gen= []
        self.prediction_gen=[]
        return {"avg_val_loss": avg_loss, 
                "accuracy" : average_em_score,
                "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]
  
    #def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None, using_native_amp=None, on_tpu=None, using_lbfgs=None):
        #optimizer.step()
        #optimizer.zero_grad()
        #self.lr_scheduler.step()
  
    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict
    
    def train_dataloader(self):   
        n_samples = self.n_obs['train']
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", num_samples=n_samples, args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=1)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler

        return dataloader

    def val_dataloader(self):
        n_samples = self.n_obs['validation']
        validation_dataset = get_dataset(tokenizer=self.tokenizer, type_path="validation", num_samples=n_samples, args=self.hparams)
        
        return DataLoader(validation_dataset, batch_size=self.hparams.eval_batch_size, num_workers=1)
    
    def test_dataloader(self):
        n_samples = self.n_obs['test']
        test_dataset = get_dataset(tokenizer=self.tokenizer, type_path="test", num_samples=n_samples, args=self.hparams)
        
        return DataLoader(test_dataset, batch_size=self.hparams.eval_batch_size, num_workers=1)
    
    def on_save_checkpoint(self, checkpoint):
        save_path = self.output_dir.joinpath(model_prefix)
        self.model.config.save_step = self.step_count
        self.model.save_pretrained(save_path)
        self.tokenizer.save_pretrained(save_path)

In [28]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # Log and save results to file
            output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

In [45]:
args_dict = dict(
    output_dir="", # path to save the checkpoints
    model_name_or_path=model_name,
    tokenizer_name_or_path=model_name,
    max_input_length=token_len,
    max_output_length=token_len,
    freeze_encoder=False,
    freeze_embeds=False,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=1,
    eval_batch_size=1,
    num_train_epochs=10,
    gradient_accumulation_steps=8,
    n_gpu=1,
    val_check_interval = 0.5, 
    n_val=-1,
    n_train=-1,
    n_test=-1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)


args_dict.update({'output_dir': "./" + model_prefix + "_final", 'num_train_epochs':50,
             'train_batch_size': 8, 'eval_batch_size': 8, 'resume_from_checkpoint' : 't5-small-512t5-small-512_ckpt_epoch_29.ckpt'})
args = argparse.Namespace(**args_dict)


## Define Checkpoint function
#checkpoint_callback = pl.callbacks.ModelCheckpoint(
#    dirpath="./t5-small-512_checkpoint", monitor="accuracy", mode="max", save_top_k=1
#)

In [30]:
model_name

't5-small'

In [46]:
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_backend='apex',
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    val_check_interval=args.val_check_interval,
    logger=wandb_logger,
    callbacks=[LoggingCallback()],
    # progress_bar_refresh_rate=0
)

model = T5BugClassifier(args)
trainer = pl.Trainer(**train_params)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [47]:
torch.cuda.empty_cache()

In [48]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loops/optimization/closure.py:36: LightningDeprecationWarning: One of the returned values {'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [49]:
torch.save(model.state_dict(), '/content/sample_data/bct5_model.pt')

In [50]:
!gsutil cp /content/sample_data/bct5_model.pt gs://robd_indep_research/Results

Copying file:///content/sample_data/bct5_model.pt [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/230.9 MiB.                                    


In [ ]:
!gsutil cp gs://robd_indep_research/Results/fn_model.pt /content/sample_data/ 

Copying gs://robd_indep_research/Results/fn_model.pt...
/ [1 files][230.9 MiB/230.9 MiB]                                                
Operation completed over 1 objects/230.9 MiB.                                    


In [51]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')
df = data.copy()
dataset = BugData(df, tokenizer, 'test', None, token_len, 3, True)
data_len = len(dataset)
print(data_len)

3291


In [ ]:
model = T5BugClassifier(args)
model.load_state_dict(torch.load('/content/sample_data/fn_model.pt'))

<All keys matched successfully>

In [52]:
loader = DataLoader(dataset, batch_size=128, shuffle=True)
it = iter(loader)

In [53]:
model.to('cuda')
dec = []
texts = []
targets = []
num = 0;
for batch in it:
  outs = model.model.generate(
            batch["source_ids"].cuda(),
            attention_mask=batch["source_mask"].cuda(),
            use_cache=True,
            decoder_attention_mask=batch['target_mask'].cuda(),
        )
  dec.extend([tokenizer.decode(ids) for ids in outs])
  texts.extend([tokenizer.decode(ids) for ids in batch['source_ids']])
  targets.extend([tokenizer.decode(ids) for ids in batch['target_ids']])
  num = num + 128
  print(num)  #track number of records

Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  torch.Size([512]) torch.Size([512])
Lens are:  to

In [54]:
import textwrap
from tqdm.auto import tqdm

In [56]:
targets2 = [a.replace("<pad>", "").replace(" ", "").replace("</s>", "") for a in targets]
dec2 = [a.replace("<pad>", "").replace("</s>", "").replace(" ", "") for a in dec]

In [59]:
print(targets2[0])
print(dec2[0])

P2high
P2normal


In [60]:
len(targets2)

3291

In [61]:
for i in range(4):
    lines = textwrap.wrap("Input Text:\n%s\n" % texts[i], width=100)
    print("\n".join(lines))
    print("\nActual Class: %s" % targets[i])
    print("\nPredicted Class from T5: %s" % dec[i])
    print("=====================================================================\n")

Input Text: classify: oops 0 panic boot pc distribution redhat hardware environment alpha dp264 smp
2 ev67 cpus 667mhz software environment problem description kernel oopses khelper boot happen 261112
boot log captured via serial console linux version 26121 gcc version 335 1 smp thu jun 23 mst 2005
booting tsunami variation dp264 using machine vector dp264 srm major options smp ev67 legacystart
magicsysrq command line rootdevsda1 consoletty0 memcluster 0 usage 1 start 0 end 341 memcluster 1
usage 0 start 341 end memcluster 2 usage 1 start end 196608 freeing pages freeing pages reserving
pages 4096k bcache detected load hit latency 21 cycles load miss latency 140 cycles smp 2 cpus
probed 3 built 1 zonelists kernel command line rootdevsda1 consoletty0 pid hash table entries 4096
order 12 131072 bytes using epoch 2000 console colour vga 80x25 dentry cache hash table entries
262144 order 8 bytes inodecache hash table entries 131072 order 7 bytes memory available kernel code
reserved data 1

In [62]:
import seaborn as sns
import matplotlib.pyplot as plt

In [63]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [64]:
confusion_matrix(targets2, dec2)

array([[  1,   0,  12,   1,  37,   0,   1,   0,   2],
       [  0,   1,   0,   4,  11,   0,   1,   2,   1],
       [  4,   0,  33,   3, 155,   0,   2,   0,  13],
       [  0,   2,   2,   1,  43,   0,   0,   2,  10],
       [  8,   1,  80,  22, 988,   0,  14,  10,  81],
       [  0,   0,   3,   0,   7,   7,  49,   3,  73],
       [  1,   0,  10,   0,  23,  11, 110,  10, 158],
       [  0,   1,   1,   1,  18,   0,  11,  23,  91],
       [  2,   0,  11,   8,  90,  12, 168,  50, 791]])

In [65]:
accuracy_score(targets2, dec2)

0.5940443634153753

In [70]:
print(classification_report(targets2, dec2))

               precision    recall  f1-score   support

   P1blocking       0.06      0.02      0.03        54
P1enhancement       0.20      0.05      0.08        20
       P1high       0.22      0.16      0.18       210
        P1low       0.03      0.02      0.02        60
     P1normal       0.72      0.82      0.77      1204
   P2blocking       0.23      0.05      0.08       142
       P2high       0.31      0.34      0.32       323
        P2low       0.23      0.16      0.19       146
     P2normal       0.65      0.70      0.67      1132

     accuracy                           0.59      3291
    macro avg       0.29      0.26      0.26      3291
 weighted avg       0.55      0.59      0.57      3291



In [67]:
target_prio = [a[:2] for a in targets2]
dec_prio = [a[:2] for a in dec2]

In [68]:
accuracy_score(target_prio, dec_prio)

0.9042844120328167

In [69]:
print(classification_report(target_prio, dec_prio))

              precision    recall  f1-score   support

          P1       0.89      0.91      0.90      1548
          P2       0.92      0.90      0.91      1743

    accuracy                           0.90      3291
   macro avg       0.90      0.90      0.90      3291
weighted avg       0.90      0.90      0.90      3291

